In [8]:
import arcpy
import pandas as pd
from arcgis.features import GeoAccessor, GeoDaskSeriesAccessor
import datetime
import os

In [ ]:
#the original table to check against
old = r"featureComparison\example_data\originalData.gdb\polygons"

#the new table to check for updates in
new = r"featureComparison\example_data\updatedData.gdb\polygons"

In [ ]:
#the column for the unique_id in the dataset
id_fld = 'unique_id'

In [ ]:
#the location where the output filegeodatabase and geodatabase table should be created
output_directory = "featureComparison\\outputs"

In [ ]:
#creates spatial dataframes using arcgis module
old_df = pd.DataFrame.spatial.from_featureclass(old)
new_df = pd.DataFrame.spatial.from_featureclass(new)

In [ ]:
#creates the output filegeodatabase and output geodatabase table
current = datetime.datetime.now().strftime("%m%d%Y%H%M%S")
out_db = f"review_{current}.gdb"
out_file = f"reviewTable_{current}"

arcpy.management.CreateFileGDB(output_directory, out_db)
arcpy.management.CreateTable(f"{output_directory}\\{out_db}", out_file)

out_table = os.path.join(output_directory, out_db, out_file)

In [ ]:
#adds summary fields to created table to populate
arcpy.management.AddField(out_table,
                          field_name = 'update_type',
                          field_type = 'TEXT')
arcpy.management.AddField(out_table, 
                          field_name = 'unique_id',
                          field_type= 'TEXT')
arcpy.management.AddField(out_table,
                          field_name = 'description',
                          field_type = 'TEXT',
                          field_length=8000)


<Result 'featureComparison\\outputs\\review_10162025094640.gdb\\reviewTable_10162025094640'>

In [ ]:
#list of output fields for use in loops
out_flds = ['update_type', 'unique_id', 'description']

In [ ]:
#checks if any original fields have been removed or any new fields have been added

old_flds = list(old_df.columns)
new_flds = list(new_df.columns)

missing = [f for f in old_flds if f not in new_flds]
new = [f for f in new_flds if f not in old_flds]

with arcpy.da.InsertCursor(out_table, out_flds) as cursor:
    if len(missing) > 0:
        for m in missing:
            up = ['DELETED FIELD', m, 'Legacy field not present in new table']
            cursor.insertRow(up)
    if len(new) > 0:
        for n in new:
            up = ['ADDED FIELD', n, 'New field added to layer']
    


In [ ]:
#checks for any individual features have been removed or added based on unique_id

old_ids = list(old_df[id_fld])
new_ids = list(new_df[id_fld])

deleted = [f for f in old_ids if f not in new_ids]
added = [f for f in new_ids if f not in old_ids]

with arcpy.da.InsertCursor(out_table, out_flds) as cursor:
    if len(deleted) > 0:
        for d in deleted:
            up = ['DELETED_FEATURE', d, 'Feature not in updated table']
            cursor.insertRow(up)
    if len(added) > 0:
        for a in added:
            up =['ADDED_FEATURE', a, 'New feature in update table']

In [ ]:
#list comprehension to get valid results existing in both tables to check for updates
existing_ids = [id for id in old_ids if id in new_ids]

valid_df = new_df.loc[new_df[id_fld].isin(existing_ids)]

valid_flds = [f for f in old_flds if f in new_flds]

In [48]:
with arcpy.da.InsertCursor(out_table, out_flds) as cursor:
    for index, row in valid_df.iterrows():
        old_row = old_df.loc[old_df[id_fld] == row[id_fld]].iloc[0]
        new_row = new_df.loc[new_df[id_fld] == row[id_fld]].iloc[0]
        
        validate = dict(old_row == new_row)
        
        if False in validate.values():
            up = ['UPDATED_FEATURE', row[id_fld], str(validate)]
            cursor.insertRow(up)

In [49]:
data = []
with arcpy.da.SearchCursor(out_table, out_flds) as cursor:
    for row in cursor:
        data.append(row)
        
df = pd.DataFrame(data, columns=out_flds)

In [1]:
type([])

list